In [ ]:
from openai import OpenAI
import base64
from pathlib import Path

client = OpenAI(base_url="http://localhost:8000/v1", api_key="not-needed")

In [ ]:
model_name = "Qwen3-0.6B"
messages = [{"role": "user", "content": "What is the capital of India?"}]

print(f"--- Sending request to model: {model_name} ---")

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0.7,
        max_tokens=500,
    )

    assistant_reply = response.choices[0].message.content
    print("\n--- Assistant's Response ---")
    print(assistant_reply)
    print("\n--------------------------\n")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

image_path = r"C:\Users\Ambik\OneDrive\Pictures\Screenshots\Screenshot 2025-05-07 224110.png"
base64_image = encode_image_to_base64(image_path)

model_name = "Qwen2.5-VL-3B-Instruct"

print(f"--- Sending image and prompt to model: {model_name} ---")

try:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is in this image? Describe it in detail."},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
                ]
            }
        ],
        max_tokens=300
    )

    assistant_reply = response.choices[0].message.content
    print("\n--- Assistant's Response ---")
    print(assistant_reply)
    print("\n--------------------------\n")

except FileNotFoundError:
    print(f"Error: The file '{image_path}' was not found. Please update the image_path variable.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
model_name = "Qwen2-Embedding-0.5B"
text_to_embed = "The quick brown fox jumps over the lazy dog."

print(f"--- Creating embedding for text: '{text_to_embed}' ---")

try:
    response = client.embeddings.create(model=model_name, input=text_to_embed)
    embedding_vector = response.data[0].embedding
    
    print(f"\nSuccessfully created embedding.")
    print(f"Vector dimensionality: {len(embedding_vector)}")
    print(f"First 5 values of the vector: {embedding_vector[:5]}")
    print("\n--------------------------------------------------\n")

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
model_name = "VoxCPM-0.5B"
text_to_speak = """
Speaker 1: I can't believe you did it again. I waited for two hours. Two hours! Not a single call, not a text. Do you have any idea how embarrassing that was, just sitting there alone?
Speaker 2: Look, I know, I'm sorry, alright? Work was a complete nightmare. My boss dropped a critical deadline on me at the last minute. I didn't even have a second to breathe, let alone check my phone.
Speaker 1: A nightmare? That's the same excuse you used last time. I'm starting to think you just don't care. It's easier to say 'work was crazy' than to just admit that I'm not a priority for you anymore.
Speaker 2: That's not fair! Of course you're a priority. You think I enjoyed being stuck in that office, drowning in spreadsheets, while knowing I was letting you down? It was stressful and I felt terrible.
Speaker 1: I just... I was really looking forward to tonight. I've had a rough week and I just wanted to see you. When you don't show, it doesn't just feel like a broken plan, it feels like I don't matter.
Speaker 2: You're right. It's not fair to you. There's no excuse. I should have found a way to let you know, even if it was just a thirty-second call. I messed up. I'm really, truly sorry. How can I make it up to you?
"""

output_file = "output_1.wav"

print(f"--- Generating speech for: '{text_to_speak}' ---")

response = client.audio.speech.create(model=model_name, input=text_to_speak, voice="alloy")

response.stream_to_file(output_file)

print(f"\n--- Success! ---")
print(f"Audio saved to: {output_file}")
print("------------------\n")

# except Exception as e:
#     print(f"An error occurred: {e}")

In [ ]:
model_name = "openai-whisper-large-v3"
audio_file_path = "output.wav"

print(f"--- Transcribing audio file: {audio_file_path} ---")

try:
    with open(audio_file_path, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(model=model_name, file=audio_file)

    print("\n--- Transcription Result ---")
    print(transcription.text)
    print("\n----------------------------\n")

except FileNotFoundError:
    print(f"Error: The file '{audio_file_path}' was not found. Please run the TTS client first or provide a valid path.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(r"D:\AI Models\Qwen3-Embedding-0.6B", device='cuda')

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-0.6B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )

queries = [
    "What is the capital of China?",
    "Explain gravity",
]
documents = [
    "The capital of China is Beijing.",
    "Gravity is a force that attracts two bodies towards each other. It gives weight to physical objects and is responsible for the movement of planets around the sun.",
]

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = model.encode(queries, prompt_name="query")
document_embeddings = model.encode(documents)

similarity = model.similarity(query_embeddings, document_embeddings)
print(similarity)
# tensor([[0.7646, 0.1414],
#         [0.1355, 0.6000]])


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = r"D:\AI Models\Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.bfloat16, device_map=device)
model.eval()

In [ ]:
# prepare the model input
prompt = "What is the capital of India?"
messages = [{"role": "user", "content": prompt}]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking = True)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(**model_inputs, max_new_tokens=32768)

output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

full_content = tokenizer.decode(output_ids, skip_special_tokens=True)

print(full_content)

# # parsing thinking content
# try:
#     # rindex finding 151668 (</think>)
#     index = len(output_ids) - output_ids[::-1].index(151668)
# except ValueError:
#     index = 0

# thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
# content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

# print("thinking content:", thinking_content)
# print("content:", content)

In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_path = r"D:\AI Models\Qwen2.5-VL-3B-Instruct"

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(model_path, dtype=torch.bfloat16, device_map=device)
processor = AutoProcessor.from_pretrained(model_path)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": r"C:\Users\Ambik\OneDrive\Pictures\Screenshots\Screenshot 2025-05-07 224110.png"},
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt").to(model.device)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(output_text[0])

In [ ]:
import soundfile as sf
from voxcpm import VoxCPM

model = VoxCPM.from_pretrained(r"D:\AI Models\TTS openbmb VoxCPM-0.5B")

In [ ]:
input_text = """ 
Speaker 1: I can't believe you did it again. I waited for two hours. Two hours! Not a single call, not a text. Do you have any idea how embarrassing that was, just sitting there alone?
Speaker 2: Look, I know, I'm sorry, alright? Work was a complete nightmare. My boss dropped a critical deadline on me at the last minute. I didn't even have a second to breathe, let alone check my phone.
Speaker 1: A nightmare? That's the same excuse you used last time. I'm starting to think you just don't care. It's easier to say 'work was crazy' than to just admit that I'm not a priority for you anymore.
Speaker 2: That's not fair! Of course you're a priority. You think I enjoyed being stuck in that office, drowning in spreadsheets, while knowing I was letting you down? It was stressful and I felt terrible.
Speaker 1: I just... I was really looking forward to tonight. I've had a rough week and I just wanted to see you. When you don't show, it doesn't just feel like a broken plan, it feels like I don't matter.
Speaker 2: You're right. It's not fair to you. There's no excuse. I should have found a way to let you know, even if it was just a thirty-second call. I messed up. I'm really, truly sorry. How can I make it up to you?
"""

wav = model.generate(
    text=input_text,
    prompt_wav_path=None,      # optional: path to a prompt speech for voice cloning
    prompt_text=None,          # optional: reference text
    cfg_value=2.0,             # LM guidance on LocDiT, higher for better adherence to the prompt, but maybe worse
    inference_timesteps=10,    # LocDiT inference timesteps, higher for better result, lower for fast speed
    normalize=True,            # enable external TN tool
    denoise=True,              # enable external Denoise tool
    retry_badcase=True,        # enable retrying mode for some bad cases (unstoppable)
    retry_badcase_max_times=3,  # maximum retrying times
    retry_badcase_ratio_threshold=6.0, # maximum length restriction for bad case detection (simple but effective), it could be adjusted for slow pace speech
)

sf.write("output_1.wav", wav, 16000)
print("saved: output.wav")

In [ ]:
import torch

def list_gpus():
    if not torch.cuda.is_available():
        print("No GPU available.")
        return
    
    device_count = torch.cuda.device_count()
    for i in range(device_count):
        name = torch.cuda.get_device_name(i)
        total_mem = torch.cuda.get_device_properties(i).total_memory / (1024 ** 3)
        reserved = torch.cuda.memory_reserved(i) / (1024 ** 3)
        allocated = torch.cuda.memory_allocated(i) / (1024 ** 3)
        free_mem = total_mem - reserved

        print(f"GPU {i}: {name}")
        print(f"  Total Memory   : {total_mem:.2f} GB")
        print(f"  Reserved Memory: {reserved:.2f} GB")
        print(f"  Allocated Memory: {allocated:.2f} GB")
        print(f"  Free Memory    : {free_mem:.2f} GB\n")

list_gpus()

In [ ]:
"""
↑ ↓ ← → 
👉 
⬆️⬇️⬅️➡️⏫ ⏬ ⏪ ⏩🔼 🔽 ◀️ ▶️↩️↪️⤴️⤵️⬆️⬇️⬅️➡️ 
✅👉 ➡️😁 ⚡✨ 🔥👌 ☑️👆
👉 👈 ☝️ 👇 🖕 ✌️ 🤞
➡️ ⬅️ ⬆️ ⬇️ ↔️ ↕️ ↪️ ↩️
⤴️ ⤵️ 🔀 🔁 🔂
➡︎ ➡︎➡︎ ➡︎➡︎➡︎ 
✅ ✔️ ☑️ 🟢
❌ ✖️ 🚫 🔴
⚠️ ❗ ❕
😀 😃 😄 😁 😆 😅 🤣 😂 🙂 🙃 😉
🤔 😎 🤩 🥳 😴 😇 🤯 😍 🥺 🙏
👍 👎 👋 🤚 ✋ 🖐️ ✊ 🤛 🤜
👐 🙌 🤲 🤝
✍️ 💪 👏
✨ 🌟 💫 🌠 ⭐
🔥 ⚡ 🌈 🌐
🎉 🎊 🎯 🎵
🔹
🔄🔃🌀
↻↺
◴ ◷ ◶ ◵ 
⟳⟲
⏳⌛
🕐 🕑 🕒 🕓 🕔 🕕 🕖 🕗 🕘 🕙 🕚 🕛
🔁🔂🔀
➰➿🔜🔛
⏩ ⏪ ⏫ ⏬
⚙️ (gear / processing)
🔧 (tool / fixing)
💫 (spinning effect)
✨ (effect / in-progress)
🌐 (network loading)
◐ ◓ ◑ ◒ (classic spinner set)
● ○ ● ○ (pulsing dots)
Loading ⏳
Loading ⌛
Loading 🔄
Loading 🌀
Loading 🔃
⏳⌛🔄🔃🔁🌀🔄⏳

"""